In [ ]:
!pip install folium leafmap

In [ ]:
import urllib
import json

import numpy as np
import pandas as pd

from shapely.geometry import shape

import folium
from folium.plugins import Draw

import leafmap

from IPython.display import JSON

## 1) Input definition

In [ ]:
start_date = '2024-12-01'
end_date = '2025-01-30'

### Define an area of interest by drawing in the map using the rectangle selection tool

In [ ]:
m = leafmap.Map(center=(47.005, 11.507), zoom=7.5)
m

In [ ]:
feat = m.draw_features
geom_dict = feat[0]['geometry']
geom = shape(geom_dict)
geom.wkt

### Alternatively upload your area of interest

## 2) Retrieve the bursts info with https request

In [ ]:
https_request = f"https://catalogue.dataspace.copernicus.eu/odata/v1/Bursts?$filter=" + urllib.parse.quote(
    f"ContentDate/Start ge {start_date}T00:00:00.000Z and ContentDate/Start le {end_date}T23:59:59.000Z and "
    f"PolarisationChannels eq 'VV' and "
    f"OData.CSC.Intersects(area=geography'SRID=4326;{geom.wkt}')"
) + "&$top=1000"

with urllib.request.urlopen(https_request) as response:
    content = response.read().decode()
bursts = json.loads(content)
JSON(bursts)

In [ ]:
bursts_uniqueTrack = {}
burstId_list = []
track_list = []
for b in bursts['value']:
    if b['RelativeOrbitNumber'] not in track_list:
        bursts_uniqueTrack[b['RelativeOrbitNumber']] = {}
        track_list.append(b['RelativeOrbitNumber'])
    if b['BurstId'] not in burstId_list:
        bursts_uniqueTrack[b['RelativeOrbitNumber']][b['BurstId']] = b['GeoFootprint']['coordinates']
        burstId_list.append(b['BurstId'])

## 3) Show on map

In [ ]:
# Initialize the map and center it in the middle of the bursts

lat, lon = [], []
for burst in bursts_uniqueTrack.values():
    for coords in burst.values():
        lat = lat + [c[1] for c in coords[0]]
        lon = lon + [c[0] for c in coords[0]]
        
m = folium.Map(
    location=[np.mean([max(lat), min(lat)]), np.mean([max(lon), min(lon)])],
    zoom_start=8
)


# Add the area of interest

if geom.geom_type == 'Point':
    folium.Marker([geom.y, geom.x]).add_to(m)

if geom.geom_type == 'Polygon':
    folium.Polygon(
        locations=[(y, x) for x, y in geom.exterior.coords],
        color='blue',
        fill=True,            
        fill_color='blue',    
        fill_opacity=0.4
    ).add_to(m)


# Add each burst grouped by track

for track, burst in bursts_uniqueTrack.items():
    
    polygon_group = folium.FeatureGroup(name=f'Track {track}')

    for burstId, polygon in bursts_uniqueTrack[track].items():

        # Create and add the first polygon to its feature group
        pol = folium.Polygon(
            locations=np.flip(np.squeeze(polygon), axis=1),
            color='red'
        )
        pol.add_to(polygon_group)
    
    polygon_group.add_to(m)


# Show the map

folium.LayerControl().add_to(m)
m

## 4) Plot calendar

In [ ]:
import calendar
import matplotlib.pyplot as plt

def display_calendar(year, month, highlighted_dates={}):

    cal = calendar.monthcalendar(year, month)
    fig, ax = plt.subplots(figsize=(4, 2))
    ax.set_xticks([])
    ax.set_yticks([])
                
    for row, week in enumerate(cal):
        for col, day in enumerate(week):
            if day == 0:
                continue
            color = highlighted_dates.get(day, "white")
            ax.add_patch(plt.Rectangle((col - 0.5, row - 0.5), 1, 1, color=color, alpha=0.6))
            ax.text(col, row, str(day), ha="center", va="center", fontsize=12, weight='bold')

    ax.set_xlim(-0.5, 6.5)
    ax.set_ylim(len(cal) - 0.5, -0.5)
    ax.set_title(calendar.month_name[month] + f" {year}")
    plt.show()

In [ ]:
date, track = [], [] 
for b in bursts['value']:
    date.append(b['BeginningDateTime'])
    track.append(b['RelativeOrbitNumber'])
df = pd.DataFrame(data={'date': date, 'track': track})
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df = df.drop_duplicates(subset=['track', 'year', 'month', 'day'])
df = df.sort_values(by='date', ascending=True)

In [ ]:
color_track = {}
fig, ax = plt.subplots(figsize=(4, len(df['track'].unique())*0.5))
for i, t in enumerate(df['track'].unique()):
    color_track[t] = f'C{i}'
    ax.add_patch(plt.Rectangle((0, -i*0.5), 0.1, 0.3, color=color_track[t], alpha=0.6))
    ax.text(0.15, -i*0.5+0.1, f'T{t}', fontsize=10)
ax.set_xlim(0, 1)
ax.set_ylim(-len(df['track'].unique())*0.5, 0.5)
ax.axis('off')
plt.show()

for year in df['year'].unique():
    df_year = df.loc[df['year'] == year, :]
    for month in df_year['month'].unique():
        h = {}
        for i, row in df_year.loc[df_year['month'] == month, :].iterrows():
            h[row['day']] = color_track[row['track']]
        display_calendar(year, month, h)

## 5) Get perpendicular baselines

In [ ]:
print(f"Select one of the available tracks: {track_list}")

In [ ]:
track = 117

Get the product names of which we need to download the metadata

In [ ]:
SAFE_image_list = []
S3_image_list = []
for b in bursts['value']:
    if b['RelativeOrbitNumber'] == track:
        if b["ParentProductName"] not in SAFE_image_list:
            SAFE_image_list.append((b["ParentProductName"]))
            S3_image_list.append((b["S3Path"].split(".SAFE")[0] + ".SAFE"))

Get the metadata from the S3 bucket (add your CDSE S3 credentials)

In [ ]:
import os
os.environ["AWS_ACCESS_KEY_ID"] = ""
os.environ["AWS_SECRET_ACCESS_KEY"] = ""
PolarisationChannels="VV"
if PolarisationChannels=="VV":
    include_pol='vv'
    exclude_pol='vh'
elif PolarisationChannels=="VH":
    include_pol='vh'
    exclude_pol='vv'

s3_endpoint = "eodata.dataspace.copernicus.eu"

for im_safe, im_s3 in zip(SAFE_image_list,S3_image_list):
    print(im_safe)
    os.system(f"s5cmd --endpoint-url \"https://{s3_endpoint}\" cp --include \"*iw1*\" --include \"*iw2*\" --include \"*iw3*\" --exclude \"*{exclude_pol}*\"  --exclude \"*.tiff\" --include \"manifest.safe\" \"s3:/\"{im_s3}\"/*\" {im_safe}/")
    os.system(f"mkdir {im_safe}/measurement")

    command = f"s5cmd --endpoint-url \"https://{s3_endpoint}\" -r 5 ls \"s3:/\"{im_s3}\"/measurement/\" | grep -o '\S\+$' | grep {include_pol} "
    result = os.popen(command).read().splitlines()

    for im in result:
        command = f"gdal_create -ot Int8 -outsize 1 1 -bands 1 -burn 0 {im_safe}/measurement/{im}"
        os.system(command)

In [ ]:
import esa_snappy as snappy
import os

os.environ["SNAP_HOME"] = "/home/mclaus@eurac.edu/snap/"
snap_home = os.environ.get("SNAP_HOME")
print(f"SNAP is installed at: {snap_home}")

# read products
products = []
for im in SAFE_image_list:
    products.append(snappy.ProductIO.readProduct(im))
master = products[0]
# import the stack operator
create_stack = snappy.jpy.get_type('eu.esa.sar.insar.gpf.coregistration.CreateStackOp')
# create_stack = snappy.jpy.get_type('org.esa.snap.core.gpf.Operator')
# 1st argument: list of products between which you want to compute the baseline
# 2nd argument: a product that will receive the baselines as new metadata
create_stack.getBaselines(products, master)
# Now there is a new piece of metadata in product one called 'Baselines'
baseline_root_metadata = master.getMetadataRoot().getElement('Abstracted_Metadata').getElement('Baselines')
# You can now display all the baselines between all master/slave configurations
master_ids = list(baseline_root_metadata.getElementNames())
for master_id in master_ids:
    slave_ids = list( baseline_root_metadata.getElement(master_id).getElementNames())
    for slave_id in slave_ids:
        print(f'{master_id}, {slave_id}')
        baseline_metadata = baseline_root_metadata.getElement(master_id).getElement(slave_id)
        for baseline in list(baseline_metadata.getAttributeNames()):
            if baseline == "Perp Baseline":
                print(f'{baseline}: {baseline_metadata.getAttributeString(baseline)}')
        print('') 